In [ ]:
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import geopandas
from shapely.geometry import Point, LineString, Polygon
from dateutil.parser import parse
import requests 
import json
from io import StringIO

# these are all things that might be currently needed 

In [ ]:
#use your own 
headers={'X-App-Token':'BLANK'}
##https://dev.socrata.com/foundry/data.sfgov.org/wr8u-xric go here to create a token 

url='https://data.sfgov.org/api/v3/views/wr8u-xric/query.json?query=SELECT%0A%20%20%60incident_number%60%2C%0A%20%20%60exposure_number%60%2C%0A%20%20%60id%60%2C%0A%20%20%60address%60%2C%0A%20%20%60incident_date%60%2C%0A%20%20%60call_number%60%2C%0A%20%20%60alarm_dttm%60%2C%0A%20%20%60arrival_dttm%60%2C%0A%20%20%60close_dttm%60%2C%0A%20%20%60city%60%2C%0A%20%20%60zipcode%60%2C%0A%20%20%60battalion%60%2C%0A%20%20%60station_area%60%2C%0A%20%20%60box%60%2C%0A%20%20%60suppression_units%60%2C%0A%20%20%60suppression_personnel%60%2C%0A%20%20%60ems_units%60%2C%0A%20%20%60ems_personnel%60%2C%0A%20%20%60other_units%60%2C%0A%20%20%60other_personnel%60%2C%0A%20%20%60first_unit_on_scene%60%2C%0A%20%20%60estimated_property_loss%60%2C%0A%20%20%60estimated_contents_loss%60%2C%0A%20%20%60fire_fatalities%60%2C%0A%20%20%60fire_injuries%60%2C%0A%20%20%60civilian_fatalities%60%2C%0A%20%20%60civilian_injuries%60%2C%0A%20%20%60number_of_alarms%60%2C%0A%20%20%60primary_situation%60%2C%0A%20%20%60mutual_aid%60%2C%0A%20%20%60action_taken_primary%60%2C%0A%20%20%60action_taken_secondary%60%2C%0A%20%20%60action_taken_other%60%2C%0A%20%20%60detector_alerted_occupants%60%2C%0A%20%20%60property_use%60%2C%0A%20%20%60area_of_fire_origin%60%2C%0A%20%20%60ignition_cause%60%2C%0A%20%20%60ignition_factor_primary%60%2C%0A%20%20%60ignition_factor_secondary%60%2C%0A%20%20%60heat_source%60%2C%0A%20%20%60item_first_ignited%60%2C%0A%20%20%60human_factors_associated_with_ignition%60%2C%0A%20%20%60structure_type%60%2C%0A%20%20%60structure_status%60%2C%0A%20%20%60floor_of_fire_origin%60%2C%0A%20%20%60fire_spread%60%2C%0A%20%20%60no_flame_spread%60%2C%0A%20%20%60number_of_floors_with_minimum_damage%60%2C%0A%20%20%60number_of_floors_with_significant_damage%60%2C%0A%20%20%60number_of_floors_with_heavy_damage%60%2C%0A%20%20%60number_of_floors_with_extreme_damage%60%2C%0A%20%20%60detectors_present%60%2C%0A%20%20%60detector_type%60%2C%0A%20%20%60detector_operation%60%2C%0A%20%20%60detector_effectiveness%60%2C%0A%20%20%60detector_failure_reason%60%2C%0A%20%20%60automatic_extinguishing_system_present%60%2C%0A%20%20%60automatic_extinguishing_sytem_type%60%2C%0A%20%20%60automatic_extinguishing_sytem_perfomance%60%2C%0A%20%20%60automatic_extinguishing_sytem_failure_reason%60%2C%0A%20%20%60number_of_sprinkler_heads_operating%60%2C%0A%20%20%60supervisor_district%60%2C%0A%20%20%60neighborhood_district%60%2C%0A%20%20%60point%60%2C%0A%20%20%60data_as_of%60%2C%0A%20%20%60data_loaded_at%60'
r=requests.get(url,headers=headers)
# this needs to be updated to use api 
# r1=requests.get('https://data.sfgov.org/resource/wr8u-xric.geojson')

In [ ]:
df_base=pd.read_json(StringIO(json.dumps(r.json())))



In [83]:
df_base

,incident_number,exposure_number,id,address,incident_date,call_number,alarm_dttm,arrival_dttm,close_dttm,city,...,number_of_sprinkler_heads_operating,action_taken_secondary,action_taken_other,fire_spread,box,ignition_factor_secondary,automatic_extinguishing_sytem_type,automatic_extinguishing_sytem_perfomance,detector_failure_reason,automatic_extinguishing_sytem_failure_reason
0,9026226,0,90262260,1670 Noe St.,2009-03-30T00:00:00.000,90890231,2009-03-30T14:31:25.000,2009-03-30T14:35:21.000,2009-03-30T14:38:39.000,SF,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,9026227,0,90262270,1 Sf Intl Airport,2009-03-30T00:00:00.000,90890232,2009-03-30T07:51:07.000,2009-03-30T07:53:53.000,2009-03-30T07:53:53.000,SFO,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,9026228,0,90262280,1 Sf Intl Airport,2009-03-30T00:00:00.000,90890233,2009-03-30T08:01:28.000,2009-03-30T08:09:51.000,2009-03-30T08:23:57.000,SFO,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,9026229,0,90262290,1 Sf Intl Airport,2009-03-30T00:00:00.000,90890235,2009-03-30T13:44:54.000,2009-03-30T13:48:54.000,2009-03-30T13:54:18.000,SFO,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,9026232,0,90262320,1 Sf Intl Airport,2009-03-30T00:00:00.000,90890240,2009-03-30T13:06:11.000,2009-03-30T13:18:11.000,2009-03-30T13:30:38.000,SFO,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
718400,9026190,0,90261900,Taylor St. / Eddy St.,2009-03-30T00:00:00.000,90890175,2009-03-30T12:37:37.000,2009-03-30T12:41:45.000,2009-03-30T12:42:06.000,SF,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
718401,9026195,0,90261950,455 Franklin St.,2009-03-30T00:00:00.000,90890182,2009-03-30T12:53:13.000,2009-03-30T12:58:16.000,2009-03-30T12:59:40.000,SF,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
718402,9026196,0,90261960,45 Mcallister St.,2009-03-30T00:00:00.000,90890185,2009-03-30T12:57:56.000,2009-03-30T13:01:54.000,2009-03-30T13:09:29.000,SF,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
718403,9026202,0,90262020,1351 Stockton St.,2009-03-30T00:00:00.000,90890201,2009-03-30T13:22:21.000,2009-03-30T13:29:51.000,2009-03-30T13:37:50.000,SF,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [84]:
df_base[df_base['neighborhood_district']=='Marina']

,incident_number,exposure_number,id,address,incident_date,call_number,alarm_dttm,arrival_dttm,close_dttm,city,...,number_of_sprinkler_heads_operating,action_taken_secondary,action_taken_other,fire_spread,box,ignition_factor_secondary,automatic_extinguishing_sytem_type,automatic_extinguishing_sytem_perfomance,detector_failure_reason,automatic_extinguishing_sytem_failure_reason
173,9026866,0,90268660,1700 - Blk Greenwich St.,2009-04-01T00:00:00.000,90910302,2009-04-01T20:04:54.000,2009-04-01T20:10:56.000,2009-04-01T20:30:05.000,SF,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
183,9026910,0,90269100,2030 Lombard St.,2009-04-01T00:00:00.000,90910352,2009-04-01T23:21:52.000,2009-04-01T23:27:15.000,2009-04-01T23:30:09.000,SF,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
288,9027279,0,90272790,Union St. / Van Ness Av.,2009-04-03T00:00:00.000,90930065,2009-04-03T07:25:01.000,2009-04-03T07:29:19.000,2009-04-03T08:15:31.000,SF,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
393,9027721,0,90277210,Lombard St. / Franklin St.,2009-04-04T00:00:00.000,90940210,2009-04-04T14:24:26.000,2009-04-04T14:28:44.000,2009-04-04T15:13:47.000,SF,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
398,9027745,0,90277450,Scott St. / Marina Bl.,2009-04-04T00:00:00.000,90940239,2009-04-04T16:07:29.000,2009-04-04T16:10:18.000,2009-04-04T16:10:49.000,SF,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
718184,9025300,0,90253000,3130 Webster St.,2009-03-27T00:00:00.000,90860203,2009-03-27T12:37:41.000,2009-03-27T12:42:44.000,2009-03-27T12:50:32.000,SF,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
718298,9025785,0,90257850,805 Marina Bl.,2009-03-29T00:00:00.000,90880004,2009-03-29T00:13:06.000,2009-03-29T00:19:32.000,2009-03-29T00:40:22.000,SF,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
718353,9025996,0,90259960,Marina Bl. / Buchanan St.,2009-03-29T00:00:00.000,90880279,2009-03-29T18:28:58.000,2009-03-29T18:36:09.000,2009-03-29T19:38:46.000,SF,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
718360,9026035,0,90260350,2539 Lombard St.,2009-03-29T00:00:00.000,90880324,2009-03-29T21:05:48.000,2009-03-29T21:10:48.000,2009-03-29T21:31:37.000,SF,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [91]:
df_base['primary_situation'].unique()

array(['745 - Alarm system sounded/no fire-accidental',
       '611 - Dispatched & canceled en route',
       '733 - Smoke detector activation/malfunction',
       '743 - Smoke detector, no fire, accidental',
       '500 - Service Call, other', '300 - Rescue, EMS call, other',
       '711 - Municipal alarm system, Street Box False',
       '740 - Unintentional alarm, other',
       '311 - Medical assist, assist EMS crew',
       '522 - Water or steam leak',
       '744 - Detector activation, no fire - accident', '511 - Lock-out',
       '411 - Gasoline or other flammable liquid spill',
       '324 - Motor vehicle accident with no injuries.',
       '118 - Trash or rubbish fire, contained',
       '151 - Outside rubbish, trash or waste fire',
       '113 - Cooking fire, confined to container',
       '142 - Brush, or brush and grass mixture fire',
       '322 - Vehicle accident with injuries',
       '353 - Removal of victim(s) from stalled elev.',
       '111 - Building fire', '323 - A

In [93]:
# df_base['primary_situation_code']=df_base['primary_situation'].apply(lambda x:x.split(" - ")[0])
# df_base['primary_situation_label']=df_base['primary_situation'].apply(lambda x:x.split(" - ")[1])

In [94]:
base_date=df_base.groupby(['neighborhood_district','primary_situation'],as_index=False)['incident_date'].nunique()
# this is the amount of unique incident_dates for each neighborhood and primary_situation
base_amount=df_base.groupby(['neighborhood_district','primary_situation'],as_index=False)['incident_number'].nunique()
# this is the amount of unique incident_numbers for each neighborhood and primary_situation

In [99]:
addresses=pd.Series(df_base[df_base['neighborhood_district']=='Marina']['address'].unique())

In [106]:
base_amount.sort_values(by='incident_number',ascending=False)

,neighborhood_district,primary_situation,incident_number
4598,Mission,"711 - Municipal alarm system, Street Box False",7149
8281,South of Market,"711 - Municipal alarm system, Street Box False",6759
8873,Tenderloin,"711 - Municipal alarm system, Street Box False",6672
1581,Financial District/South Beach,"700 False alarm or false call, other",4497
8870,Tenderloin,"700 False alarm or false call, other",4133
...,...,...,...
32,Bayview Hunters Point,133 - Rail vehicle fire,1
3168,Japantown,443 - Light ballast breakdown,1
19,Bayview Hunters Point,121 - Fire in mob. home used as residence,1
10106,Western Addition,814 - Lightning strike (no fire),1


In [107]:
base_date.sort_values(by='incident_date',ascending=False)

,neighborhood_district,primary_situation,incident_date
4598,Mission,"711 - Municipal alarm system, Street Box False",3050
8281,South of Market,"711 - Municipal alarm system, Street Box False",2852
8873,Tenderloin,"711 - Municipal alarm system, Street Box False",2842
1581,Financial District/South Beach,"700 False alarm or false call, other",2434
8870,Tenderloin,"700 False alarm or false call, other",2352
...,...,...,...
2681,Inner Richmond,371 - Electrocution or potential electrocution,1
11,Bayview Hunters Point,"115 Incinerator overload or malfunction, fire ...",1
36,Bayview Hunters Point,136 - Self-propelled motor home or RV,1
33,Bayview Hunters Point,133 Rail vehicle fire,1


In [110]:
base_date[base_date['primary_situation']=='700 - False alarm or false call, other'].sort_values(by='incident_date',ascending=False)

,neighborhood_district,primary_situation,incident_date
8869,Tenderloin,"700 - False alarm or false call, other",1632
1580,Financial District/South Beach,"700 - False alarm or false call, other",1614
4594,Mission,"700 - False alarm or false call, other",1217
8277,South of Market,"700 - False alarm or false call, other",1008
5115,Nob Hill,"700 - False alarm or false call, other",844
10058,Western Addition,"700 - False alarm or false call, other",730
255,Bayview Hunters Point,"700 - False alarm or false call, other",725
5616,North Beach,"700 - False alarm or false call, other",667
1020,Chinatown,"700 - False alarm or false call, other",644
6644,Pacific Heights,"700 - False alarm or false call, other",558


In [111]:
base_amount[base_amount['primary_situation']=='700 - False alarm or false call, other'].sort_values(by='incident_number',ascending=False)

,neighborhood_district,primary_situation,incident_number
1580,Financial District/South Beach,"700 - False alarm or false call, other",2537
8869,Tenderloin,"700 - False alarm or false call, other",2417
4594,Mission,"700 - False alarm or false call, other",1622
8277,South of Market,"700 - False alarm or false call, other",1287
5115,Nob Hill,"700 - False alarm or false call, other",1058
255,Bayview Hunters Point,"700 - False alarm or false call, other",902
10058,Western Addition,"700 - False alarm or false call, other",884
5616,North Beach,"700 - False alarm or false call, other",798
1020,Chinatown,"700 - False alarm or false call, other",789
6644,Pacific Heights,"700 - False alarm or false call, other",684
